In [1]:
from pickletools import uint8
import matplotlib.pyplot as plt
import numpy as np
import sys
import time
import tkinter as tk
from tkinter import ttk
import customtkinter as ctk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from PIL import Image, ImageTk
from data_organizer_Kfall import DataOrganizer
from sklearn.model_selection import train_test_split
import serial
import time
import struct
import threading
import queue
import re

In [2]:
# load data from ./saved_data
X_test = np.load('./saved_data/X_test.npy')
y_test = np.load('./saved_data/y_test.npy')
# shuffle the X_test and y_test in the same order
ran_index = np.random.permutation(len(X_test))
X_test = X_test[ran_index]
y_test = y_test[ran_index]
# get the length of the test data

test_len = len(X_test)
y_pred = np.random.randint(2, size=test_len)
print(X_test.shape)
print(y_test.shape)


(190, 50, 6)
(190,)


In [4]:
class InferenceGUI:

    def __init__(self, X_test, y_test, y_pred, 
                 port='COM3', baudrate=115200, MCU = 'STM32', confidence = 0.95):
        
        self.MCU = MCU
        self.port = port
        self.baudrate = baudrate
        self.X_test = X_test
        self.y_test = y_test
        #self.y_pred = y_pred
        self.test_data = iter(zip(X_test, y_test))
        self.total = 0
        self.correct = 0
        self.confidence = confidence
        self.pred = 0
        self.queue = queue.Queue()
        self.mode = -1
        self.task = None
        self.serial_in_use = False
        self.setup_UI()

    def setup_UI(self):

        self.root = tk.Tk()
        self.root.geometry("1024x768")
        self.root.title("Fall Detection")
        self.root.resizable(True, True)
        self.pred_cache = tk.StringVar()
        if self.MCU == 'STM32':

            self.read_serial_data()
            self.root.protocol("WM_DELETE_WINDOW", self.on_close)
            self.root.mainloop()
        else:
            #TODO: MAXIM MCU
            pass

    def on_close(self):
        # Close the serial connection
        if hasattr(self, 'ser') and self.ser.is_open:
            self.ser.close()
        # Destroy the window
        self.root.destroy()

    def read_serial_data(self):

        self.ser = serial.Serial(port=self.port, baudrate=self.baudrate, timeout=3)   
        # flush the serial port
        self.ser.flush()
        self.ser.reset_input_buffer()
        self.ser.reset_output_buffer()
        self.welcome_page()

        # Start a new thread to update the page
        threading.Thread(target=self.serial_daemon, daemon=True).start()
    
    
    # def host_mcu_handshake(self):
    #     # Sending the host request
    #     self.ser.reset_input_buffer()
    #     self.ser.write(b'Connect\r\n')
        
    #     line = self.ser.readline().decode('utf-8').strip()
    #     if line == 'Echo':
    #         return True
    #     else:
    #         return False
        #time.sleep(0.1)
    
    

    def test_once_on_MCU(self):
        x, y = next(self.test_data)
        # shape of x needs to be (1, 50, 6), and type needs to be float32
        x = np.expand_dims(x, axis=0).astype(np.float32)
        #x_bytes = struct.pack(f'{x.size}f', *x.flatten())
        x_bytes = x.tobytes()
        # self.send_data_to_MCU(x_bytes)
        #print(x_bytes.shape) # (1, 50, 6)
        while self.serial_in_use:
            pass
        self.serial_in_use = True
        self.ser.reset_input_buffer()
        self.ser.write(b'Connect\r\n')
        line = self.ser.readline().decode('utf-8').strip()
        if line == 'Echo':
            self.ser.write(x_bytes)
            line = self.ser.readline().decode('utf-8').strip() # z.B.-127,127
            self.serial_in_use = False
            if line == 'Data received.':
                print('Data sent to MCU.')
            else:
                print('Data sending failed.')
                self.task = None
                return
        else:
            print('Handshake failed.')
            self.serial_in_use = False
            self.task = None
            return
        try:
            (output1, output2) = self.queue.get(timeout=3)
        except:
            print('No inference result received.')
            self.task = None
            return
        if output1 >= output2:
            y_pred = 0
        else:
            y_pred = 1
        self.total += 1
        if y == y_pred:
            self.correct += 1

        if y == 0:
            self.label_text.set('Not Falling')
            self.left_frame.config(bg='green')
            self.label.config(background='green')
            self.label2.config(background='green')
        else:
            self.label_text.set('Falling')
            self.left_frame.config(bg='red')
            self.label.config(background='red')
            self.label2.config(background='red')
        if y_pred == 0:
            self.prediction_text.set('Not Falling')
            self.right_frame.config(bg='green')
            self.prediction_label.config(background='green')
            self.prediction_label2.config(background='green')
        else:
            self.prediction_text.set('Falling')
            self.right_frame.config(bg='red')
            self.prediction_label.config(background='red')
            self.prediction_label2.config(background='red')
        self.task = None
        #time.sleep(0.1)
    

    def serial_daemon(self):
        while True:
            while self.serial_in_use:
                pass
            self.serial_in_use = True
            if self.ser.in_waiting:
                line = self.ser.readline().decode('utf-8').strip()
                if line.find('Inference completed') != -1:
                    match = re.search(r"output=\[(.*), (.*)\], elapsed time: (\d+)", line)
                    if match:
                        output1 = int(match.group(1))
                        output2 = int(match.group(2))
                        elapsed_time = int(match.group(3))
                        print(f'Inference output received, output1: {output1}, output2: {output2}, inference delay: {elapsed_time}us.')
                        if self.mode == 1:
                            self.queue.put((output1, output2))
                        elif self.mode == 2:
                            if output1 >= output2 or output2 <= self.confidence*127:
                            # if output1>=output2:
                                self.pred_label.config(text="You are safe!")
                            else:
                                self.pred_label.config(text="You are Falling!!! Call 112!!!")
                        # if output1 >= output2:
                        #     y = 0
                        # else:
                        #     y = 1
                        # if self.mode == 2:
                        #     if y == 0:
                        #         self.pred_label.config(text="You are safe!")
                        #     else:
                        #         self.pred_label.config(text="You are Falling!!! Call 112!!!")
                        # elif self.mode == 1:
                        #     self.total += 1
                            # if y == y_pred:
                            #     self.correct += 1

                            # if y == 0:
                            #     self.label_text.set('Not Falling')
                            #     self.left_frame.config(bg='green')
                            #     self.label.config(background='green')
                            #     self.label2.config(background='green')
                            # else:
                            #     self.label_text.set('Falling')
                            #     self.left_frame.config(bg='red')
                            #     self.label.config(background='red')
                            #     self.label2.config(background='red')
                            # if y_pred == 0:
                            #     self.prediction_text.set('Not Falling')
                            #     self.right_frame.config(bg='green')
                            #     self.prediction_label.config(background='green')
                            #     self.prediction_label2.config(background='green')
                            # else:
                            #     self.prediction_text.set('Falling')
                            #     self.right_frame.config(bg='red')
                            #     self.prediction_label.config(background='red')
                            #     self.prediction_label2.config(background='red')
                # mode 1: inference on PC
                elif line == 'Mode 1 selected.':
                    self.mode = 1
                    # if self.task is not None, then stop the thread
                    self.show_inference_from_PC_page()

                # mode 2: inference on MCU
                elif line == 'Mode 2 selected.':
         
                    self.mode = 2
                    # if self.task is not None, then stop the thread
                    self.show_inference_from_MCU_page()
                    # output = self.ser.readline().decode('utf-8').strip().split(',') # z.B.-127,127
                    # output0 = output[0]
                    # output1 = output[1]
                    # if output0 > output1:
                    #     self.pred = 0
                    #     #self.pred_cache.set("You are safe!")
                    # else:
                    #     self.pred = 1
                        #self.pred_cache.set("You are Falling!!! Call 112!!!")

                # mode 0: idle
                elif line == 'Mode 0 selected.':
           
                    self.mode = 0
                    self.show_idle_page()
                    
                elif line.find('Inference failed') != -1:
                    print('Inference failed on MCU.')

            self.serial_in_use = False
            self.root.update()
            
    def clear_page(self):
        for widget in self.root.winfo_children():
            widget.destroy()
    
    def welcome_page(self):

        self.clear_page()

        self.welcome_label = ttk.Label(self.root, text="Please press the switch button \nto switch mode..", font=("Arial", 40))
        self.welcome_label.pack(expand=True)


    def show_inference_from_PC_page(self):
        # Clear the current content
        self.clear_page()

        # reset the self.task when going back to this page
        self.task = None

        # Central frame for mode selection
        self.mode_selection_frame = ttk.Frame(self.root)
        self.mode_selection_frame.pack(expand=True)

        # Label for mode selection
        self.mode_label = ttk.Label(self.mode_selection_frame, text="Select prediction mode:", font=("Arial", 40))
        self.mode_label.pack(pady=(10, 20))  # Adding some vertical padding

        # Frame to hold buttons
        self.button_frame = ttk.Frame(self.mode_selection_frame)
        self.button_frame.pack(pady=10)

        # Create a style
        style_button = ttk.Style()
        # Configure the style
        style_button.configure('TButton', font=('Arial', 25))  # Increase the font size

        # Auto mode button
        self.auto_mode_button = ttk.Button(self.button_frame, text="Auto", command=self.enter_auto_mode, style='TButton')
        self.auto_mode_button.pack(side=tk.LEFT, padx=10)

        # Manual mode button
        self.manual_mode_button = ttk.Button(self.button_frame, text="Manual", command=self.enter_manual_mode, style='TButton')
        self.manual_mode_button.pack(side=tk.RIGHT, padx=10)
    

    def test_once_on_MCU_auto(self):
        
        for x, y in zip(self.X_test, self.y_test):
            
            if self.task is None:
                break

            # shape of x needs to be (1, 50, 6), and type needs to be float32
            x = np.expand_dims(x, axis=0).astype(np.float32)
            #x_bytes = struct.pack(f'{x.size}f', *x.flatten())
            x_bytes = x.tobytes()
            # self.send_data_to_MCU(x_bytes)
            #print(x_bytes.shape) # (1, 50, 6)
            while self.serial_in_use:
                pass
            self.serial_in_use = True
            self.ser.reset_input_buffer()
            self.ser.write(b'Connect\r\n')
            line = self.ser.readline().decode('utf-8').strip()
            if line == 'Echo':
                self.ser.write(x_bytes)
                line = self.ser.readline().decode('utf-8').strip() # z.B.-127,127
                if line == 'Data received.':
                    print('Data sent to MCU.')
                else:
                    print('Data sending timeout.')
            else:
                print('Handshake timeout')
            self.serial_in_use = False
            (output1, output2) = self.queue.get(timeout=3)
            if output1 >= output2:
                y_pred = 0
            else:
                y_pred = 1
            self.total += 1
            if y == y_pred:
                self.correct += 1
            self.accuracy = self.correct/self.total*100
            self.accuracy = round(self.accuracy, 2)
            # self.idx_label.config(text=f"Infering number {self.total} out of {self.num_total} set of data\n Accuracy: {self.accuracy}")
            self.idx_label.config(text=f"Infering number {self.total} out of {self.num_total} set of data")
            self.accuracy_label.config(text=f"Accuracy: {self.accuracy}%")
            
            print(f'Ground truth: {y}, prediction: {y_pred}')
            
        



    def enter_auto_mode(self):
        # Clear the current content
        self.clear_page()

        # reset the correct and total
        self.correct = 0
        self.total = 0
        self.current_idx = 0
        self.accuracy = 0
        self.num_total = len(self.y_test)
        # reinitialize the queue
        self.queue = queue.Queue()
        # TODO: Create the content for the self.correct, self.total and self.current_idx. They are all put inthe middle of the page
        
        # Create labels for the self.correct, self.total and self.current_idx
        # self.idx_label = ttk.Label(self.root, text=f"Infering number {self.total} out of {self.num_total} set of data\n Accuracy: {self.accuracy}", font=("Arial", 40, 'bold'))
        self.idx_label = ttk.Label(self.root, text=f"Infering number {self.total} out of {self.num_total} set of data", font=("Arial", 40, 'bold'))
        self.idx_label.grid(row=0, column=1)
        self.idx_label.place(relx=0.5, rely=0.5, anchor='center')
        # Create a label for the accuracy
        self.accuracy_label = ttk.Label(self.root, text=f"Accuracy: {self.accuracy}%", font=("Arial", 40, 'bold'))
        self.accuracy_label.place(relx=0.5, rely=0.6, anchor='center')

        # crate a frame 
        if self.task == None:
            self.task = threading.Thread(target=self.test_once_on_MCU_auto)
            self.task.start()
            

        # Back button
        self.back_button = ttk.Button(self.root, text="Back", command=self.show_inference_from_PC_page, style='TButton')
        self.back_button.pack(pady=10)
        self.back_button.place(relx=0.5, rely=1, anchor='s')

    def enter_manual_mode(self):
        self.clear_page()

        self.top_frame = tk.Frame(self.root)
        self.top_frame.pack(side='top', fill='both', expand=True)
        # Create a frame for the left half of the window
        self.left_frame = tk.Frame(self.top_frame, bd=2, relief='solid', bg='white')
        self.left_frame.pack(side='left', fill='both', expand=True)
        self.left_frame.pack_propagate(False)  # Prevent the frame from changing size

        # Create a frame for the right half of the window
        self.right_frame = tk.Frame(self.top_frame, bd=2, relief='solid', bg='white')
        self.right_frame.pack(side='left', fill='both', expand=True)
        self.right_frame.pack_propagate(False)  # Prevent the frame from changing size

        # Back button
        self.back_button = ttk.Button(self.root, text="Back", command=self.show_inference_from_PC_page, style='TButton')
        self.back_button.pack(side=tk.LEFT, padx=120, pady=10)

        # Next button
        self.next_button = ttk.Button(self.root, text="Next Prediction", command=self.on_button_click, style='TButton')
        self.next_button.pack(side=tk.RIGHT, padx=120, pady=10)

  
        self.label = tk.Label(self.left_frame, text='Ground Truth Label:', font=("Helvetica", 24), bg='white')
        self.label.pack()

        self.label_text = tk.StringVar()
        self.label2 = tk.Label(self.left_frame, textvariable=self.label_text, font=("Helvetica", 50), bg='white')
        self.label2.place(relx=0.5, rely=0.5, anchor='center')

        self.prediction_label = tk.Label(self.right_frame, text='Prediction:', font=("Helvetica", 24), bg='white')
        self.prediction_label.pack()

        # Create a label to display the prediction
        self.prediction_text = tk.StringVar()
        self.prediction_label2 = tk.Label(self.right_frame, textvariable=self.prediction_text, font=("Helvetica", 50), bg='white')
        self.prediction_label2.place(relx=0.5, rely=0.5, anchor='center')

    def on_button_click(self):
        # print('on_button_click')
        #y_pred = self.ser.readline().decode('utf-8').strip().split(',')# z.B.-127,127

        # take the highest value
        #y_pred = max(map(int, y_pred))
        if self.task == None:
            self.task = threading.Thread(target=self.test_once_on_MCU)
            self.task.start()
        # if self.host_mcu_handshake():
        #     # write data to MCU
        #     if self.send_data_to_MCU(x_bytes):
        #         print('Data sent to MCU.')
        #     else:
        #         print('Data sending timeout.')
        # else:
        #     print('Handshake timeout')
        #     self.total += 1
        #     if y == y_pred:
        #         self.correct += 1

        #     if y == 0:
        #         self.label_text.set('Not Falling')
        #         self.left_frame.config(bg='green')
        #         self.label.config(background='green')
        #         self.label2.config(background='green')
        #     else:
        #         self.label_text.set('Falling')
        #         self.left_frame.config(bg='red')
        #         self.label.config(background='red')
        #         self.label2.config(background='red')
        #     if y_pred == 0:
        #         self.prediction_text.set('Not Falling')
        #         self.right_frame.config(bg='green')
        #         self.prediction_label.config(background='green')
        #         self.prediction_label2.config(background='green')
        #     else:
        #         self.prediction_text.set('Falling')
        #         self.right_frame.config(bg='red')
        #         self.prediction_label.config(background='red')
        #         self.prediction_label2.config(background='red')
        # except StopIteration:
            # self.label_text.set('No more test data')


    
    # def show_inference_from_MCU_page(self):
    #     # Clear the current content
    #     self.clear_page()

        

    #     self.pred_label = ttk.Label(self.root, font=("Arial", 50, 'bold'))
    #     self.pred_label.place(relx=0.5, rely=0.5, anchor='center')


    #     self.pred_label.pack(expand=True)
    #     while True:
    #         output = self.ser.readline().decode('utf-8').strip().split(',') # z.B.-127,127
    #         output0 = int(output[0])
    #         output1 = int(output[1])
  
    #         if output0 >= output1 or output1 <= self.confidence*127:
    #             self.pred_label.config(text="You are safe!")
    #         else:
    #             self.pred_label.config(text="You are Falling!!!\nCall 112!!!")
    #             # hold for 3 seconds
    #             #time.sleep(3)
    #             print(output)
        

    def show_inference_from_MCU_page(self):
        # Clear the current content
        self.clear_page()

        # Create the label with the updated text
        self.pred_label = ttk.Label(self.root, font=("Arial", 50, 'bold'))
        self.pred_label.place(relx=0.5, rely=0.5, anchor='center')
        #output = self.ser.readline().decode('utf-8').strip().split(',') # e.g., "-127,127"
        # try:
        #     # refresh every 50ms
        #     self.update_output()
        #     # output0 = int(output[0])
        #     # output1 = int(output[1])
        #     # #self.update_output(output0, output1)
        #     # if output0 >= output1 or output1 <= self.confidence * 127:
        #     #     self.pred_label.config(text="You are safe!")
        #     # else:
        #     #     self.pred_label.config(text="You are Falling!!! Call 112!!!")
        # except (ValueError, IndexError):
        #     pass  # Handle errors if output is not as expected

    
    # def update_output(self):
    #     output = self.ser.readline().decode('utf-8').strip() # e.g., "-127,127"
        
    #     if output != 'Mode 2 selected.' \
    #             and output != 'Mode 1 selected.' \
    #             and output != 'Mode 0 selected.' \
    #             and output != 'Echo' \
    #             and output != 'Data received.': 
    #         output = output.split(',') # e.g., "-127,127"
    #         output0 = int(output[0])
    #         output1 = int(output[1])
    #         #if output0 >= output1 or output1 <= self.confidence * 127:
    #         if output0 >= output1:
    #             self.pred_label.config(text="You are safe!")
    #         else:
    #             self.pred_label.config(text="You are Falling!!! Call 112!!!")
            
    #         self.root.after(50, self.update_output)
   

    def show_idle_page(self):
        # Clear the current content
        self.clear_page()

        # Create new content for idle page
        self.idle_label = ttk.Label(self.root, text="You are in Sleep Mode\nZZZ....\n\nPress the switch button \nto switch mode.", font=("Arial", 40))
        self.idle_label.pack(expand=True)


inference_gui = InferenceGUI(X_test, y_test, y_pred, 
                             port='/dev/ttyACM0', baudrate=115200, MCU = 'STM32', confidence = 0.7)
# inference_gui = InferenceGUI(X_test, y_test, y_pred, 
#                              port='/dev/tty.usbmodem103', baudrate=115200, MCU = 'STM32', confidence = 0.7)


Data sent to MCU.
Inference output received, output1: 108, output2: -108, inference delay: 28386us.
Ground truth: 0, prediction: 0
Data sent to MCU.
Inference output received, output1: -97, output2: 97, inference delay: 28385us.
Ground truth: 1, prediction: 1
Data sent to MCU.
Inference output received, output1: 127, output2: -128, inference delay: 28381us.
Ground truth: 0, prediction: 0
Data sent to MCU.
Inference output received, output1: -128, output2: 127, inference delay: 28384us.
Ground truth: 1, prediction: 1
Data sent to MCU.
Inference output received, output1: 127, output2: -128, inference delay: 28385us.
Ground truth: 0, prediction: 0
Data sent to MCU.
Inference output received, output1: -127, output2: 127, inference delay: 28383us.
Ground truth: 1, prediction: 1
Data sent to MCU.
Inference output received, output1: 123, output2: -123, inference delay: 28382us.
Ground truth: 1, prediction: 0
Data sent to MCU.
Inference output received, output1: -126, output2: 126, inference d

Exception in thread Thread-5 (serial_daemon):
Traceback (most recent call last):
  File "/home/liyinrong/miniconda3/envs/tensorflow-dev/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/liyinrong/miniconda3/envs/tensorflow-dev/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1625/3531826333.py", line 145, in serial_daemon
  File "/home/liyinrong/miniconda3/envs/tensorflow-dev/lib/python3.10/site-packages/serial/serialposix.py", line 549, in in_waiting
    s = fcntl.ioctl(self.fd, TIOCINQ, TIOCM_zero_str)
TypeError: argument must be an int, or have a fileno() method.


In [11]:
classes = ['You are Falling!!! Call 112!!!', 'Your are safe!']
mode = [0, 1, 2] # 0: Sleep, 1: Inference on PC, 2: Inference on MCU
print('mode: ', mode)

# ser = serial.Serial(port='/dev/tty.usbmodem103', baudrate=115200, timeout=3)
ser = serial.Serial(port='/dev/ttyACM0', baudrate=115200, timeout=3)
# flush the serial port
ser.flush()
ser.flushInput()
ser.flushOutput()

mode:  [0, 1, 2]


In [9]:
ser.close()

NameError: name 'ser' is not defined

In [12]:
#print(X_test[0].shape)
x = np.expand_dims(X_test[0], axis=0).astype(np.float32)
# Pack each float value in the flattened array
x_bytes = x.tobytes()
print(x_bytes)


b"}?u?o\x12\x83<h\x91\r?\xa4\x02\x8f@]\x18\xa4\xc1\x10\x04\\Ashq?\xa6\x9bD;\\\x8f\x02?TYX@\xdf\x98\xbf\xc1\xdc\x03QA\x1f\x85k?\n\xd7\xa3\xbb\xa2E\xf6>\x84X,@\xcc\xae\xdf\xc1~.OA\x14\xaeg?1\x08\xac\xbc{\x14\xee>\xb8X7@\xaf\x99\xeb\xc1\x8b\xeeQA\xa6\x9bd?\x02+\x07\xbd\n\xd7\xe3>\x88Yc@\ro\xed\xc1\x98\xaeTA\xd3Mb?\x1b/\xdd\xbcT\xe3\xe5>\xf0Yy@\xc6N\xde\xc1\xd2\xd8<Ad;_?B`\xe5;\x8bl\xe7>\xa4\x02\x8f@k\x0e\xcb\xc1\x1cX\x16Aw\xbe_?\xbct\x93<\xc1\xca\xe1>\xb2.\xda@W\xee\xc6\xc1*\x84\xe1@\n\xd7c?\xa6\x9b\xc4;u\x93\xd8>)\x18\x19A\xbc#\xca\xc10\xae\xbe@\x02+g?\xbct\x13\xbc\x1b/\xdd>@\x030A^N\xc8\xc1:\xd9\xd2@\x17\xd9n?\xbct\x13\xbc\xd3M\xe2>\xab\x984A\xf9\x18\xc5\xc1\xe8W\x0bA\x8f\xc2u?\x00\x00\x00\x00\xe5\xd0\xe2>0\xae>A\xc9\xe3\xcc\xc10\xae>A\x87\x16y?\n\xd7\xa3;\xd3M\xe2>\xb8X7A\xbf\xee\xdc\xc1\xa2\xd9hA\xc5 \x80?B`\xe5;R\xb8\xde>\x16.9Ar\xa4\xf0\xc1\xfd\x19|A\xd3M\x82?o\x12\x03;\x9a\x99\xd9>\x19\xc3'A]\xfd\x11\xc23\r\x89A/\xdd\x84?\xa6\x9bD;`\xe5\xd0>\xa4\x02\x0fA\xc2M'\xc2\x16\xf8\x94A\x98n

In [13]:
while True:
    if ser.in_waiting > 0:
        line = ser.readline().decode('utf-8').strip()
        #print(line) # output[0]=17 output[1]=-17
        
        if line == 'Mode 2 selected.':
            #print("Fall detection with data sent from MCU.")
            output0 = line[0]
            output1 = line[1]
        # if  'output[' in line:
        #     outputs = line.split()
        #     output0 = int(outputs[0].split('=')[1])
        #     output1 = int(outputs[1].split('=')[1])
            # print the highest probability between output0 and output1
            # if output0 > output1:
            #     print('You are safe!')
            # else:
            #     print('You are falling!!! Call 112!!!')
        if line == 'Mode 1 selected.':
            
            ser.write(b'Connect\r\n')
            #ser.write(x_bytes)
            ser.write(x_bytes)
            print('sssssssssssssssss')
            if line != 'Echo' and line != 'Mode 1 selected.':
                print(line)
            
        elif line == 'Mode 0 selected.':
            print("Sleep mode.")
      
            

sssssssssssssssss
Sleep mode.


KeyboardInterrupt: 